This notebook is associated with the paper "The relative class number one problem for function fields, II" by K.S. Kedlaya. It runs in SageMath (tested using version 9.5rc4) and Magma (tested using version 2.26-9); it also requires the `pandas` and `openpyxl` libraries (`sage --pip install openpyxl/pandas`).

In this notebook, we make computations as part of the proof that for $q=2$, there are no non-Galois covers with relative class number 1. This depends on the exhaustion over Weil polynomials performed in "The relative class number one problem for function fields, I".

# Setup

In [1]:
load("../Shared/weil_poly_utils.sage")
load("splitting_sequences.sage")

In [2]:
P.<T> = QQ[]

Read data about the exhaustion from an Excel spreadsheet.

In [3]:
import pandas

In [4]:
df = pandas.read_excel('../Shared/polys.xlsx')
assert 'Cyclic' in df
print(list(df))

['Unnamed: 0', 'd', 'g', "g'", 'Label of J(C)', '#J(C)(F_2)', '#J(C)(F_4)', 'Counts of C', "Counts of C'", 'Cyclic']


In [5]:
candidates = {}
for i in range(len(df)):
    r = df.iloc[i]
    d = r["d"]
    g = r["g"]
    g1 = r["g'"]
    if (d,g,g1) not in candidates:
        candidates[d,g,g1] = []
    counts1 = eval(r["Counts of C"])
    counts2 = eval(r["Counts of C'"])
    ordj2 = r["#J(C)(F_2)"]
    ordj4 = r["#J(C)(F_4)"]
    candidates[d,g,g1].append((counts1, counts2, ordj2, ordj4))

Check that at least one degree-1 place lifts to a degree-$d$ place (Lemma 2.3).

In [6]:
for (d, g, g1) in candidates:
    if d > 2:
        for (a,b,_,_) in candidates[d,g,g1]:
            ct1 = place_count_from_point_count(a, g)
            ct2 = place_count_from_point_count(b, g1)
            if not (ct1[0] > sum(ct2[i] for i in range((d-1)//2)) + (0 if d%2 else 1/2*ct2[d//2-1])):
                print(d,g,g1,ct1,ct2)
                assert (ct2[0] == 0 and ct2[2] == 0)

5 2 6 [5, 2] [0, 5, 0, 0, 5, 10]


Verify some statements about the data for $d=2$ (Remark 2.4).

In [7]:
for (a,b,_,_) in candidates[2,2,3]:
    assert (a[0], a[1], b[0]) in [(2,8,0), (4,8,2)]

In [8]:
for (a,b,_,_) in candidates[2,3,5]:
    assert b[0] <= 2
    if a[0] == 4:
        assert a[1] == 8

In [9]:
for (a,b,_,_) in candidates[2,3,6]:
    assert a[0] >= 4

In [10]:
for (a,b,_,_) in candidates[2,4,7]:
    assert a[0] >= 3

In [11]:
for (a,b,_,_) in candidates[2,5,9]:
    assert a[0] >= 2
    if a[0] == 2:
        assert b[1] == 4

# Case $d=3$ (Lemma 5.2)

Subcase $(g,g') = (4,10)$.

In [12]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,4,10]]
l

[((6, 6), (0, 0)),
 ((6, 6), (0, 0)),
 ((7, 7), (0, 0)),
 ((7, 9), (0, 2)),
 ((8, 8), (0, 0))]

In [13]:
for ct1, ct2 in l:
    for s in splitting_sequences(ct1, ct2, 3):
        trB = splittings_to_quadratic_resolvent(ct1, s)
        assert trB[0] <= -7 or (trB[0] == -6 and trB[1] == -6)
        assert not weil_polys_from_traces(P, 6, 2, trB)

Subcase $(g,g') = (3,7)$, $\#C'(\mathbb{F}_2) > 1$.

In [14]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,3,7] if b[0] > 1]
l

[((6, 10), (2, 2))]

In [15]:
assert all(not splitting_sequences(ct1, ct2, 3) for ct1, ct2 in l)

Subcase $(g,g') = (3,7)$, $\#C'(\mathbb{F}_2) = 1$.

In [16]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,3,7] if b[0] == 1 and b[1] >= 3]
l

[((5, 9), (1, 3))]

In [17]:
for ct1, ct2 in l:
    tmp = splitting_sequences(ct1, ct2, 3)
    print(tmp)
    for s in tmp:
        trB = splittings_to_quadratic_resolvent(ct1, s)
        print(trB)
        assert not weil_polys_from_traces(P, 4, 2, trB)

[[([3], [3], [3], [3], [2, 1]), ([3], [3])]]
(-3, -9)


Subcase $(g,g') = (3,7)$, $\#C'(\mathbb{F}_2) = 0$, $\#C'(\mathbb{F}_4) > 0$.

In [18]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,3,7] if b[0] == 0 and b[1] > 0]
l

[((4, 8), (0, 2)), ((4, 10), (0, 2)), ((4, 12), (0, 6))]

In [19]:
for ct1, ct2 in l:
    tmp = splitting_sequences(ct1, ct2, 3)
    print(tmp)
    for s in tmp:
        trB = splittings_to_quadratic_resolvent(ct1, s)
        print(trB)
        assert (not weil_polys_from_traces(P, 4, 2, trB) or ct1[1] != 8)

[[([3], [3], [3], [3]), ([3], [2, 1])]]
(-4, -4)
[[([3], [3], [3], [3]), ([3], [3], [2, 1])]]
(-4, -6)
[[([3], [3], [3], [3]), ([3], [3], [3], [1, 1, 1])], [([3], [3], [3], [3]), ([3], [2, 1], [2, 1], [2, 1])]]
(-4, -12)
(-4, 0)


Subcase $(g,g') = (3,7)$, $\#C'(\mathbb{F}_2) = 0$, $\#C'(\mathbb{F}_4) = 0$.

In [20]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,3,7] if b[0] == 0 and b[1] == 0]
l

[((2, 8), (0, 0)),
 ((3, 7), (0, 0)),
 ((3, 7), (0, 0)),
 ((4, 6), (0, 0)),
 ((4, 6), (0, 0)),
 ((4, 8), (0, 0)),
 ((4, 8), (0, 0)),
 ((4, 8), (0, 0)),
 ((4, 8), (0, 0))]

In [21]:
for ct1, ct2 in l:
    assert not weil_polys_from_traces(P, 4, 2, ct1)

Subcase $(g,g') = (2,4)$.

In [22]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,2,4]]
l

[((1, 9), (0, 6)),
 ((3, 3), (0, 4)),
 ((3, 9), (0, 10)),
 ((4, 6), (0, 6)),
 ((5, 5), (1, 5))]

In [23]:
assert all(trace_from_weil_poly(u, 2)[1] == -3 for u in weil_polys_from_traces(P, 2, 2, [-1]))

Subcase $(g,g') = (2,6)$.

In [24]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[3,2,6]]
assert all(ct1[0] >= 3 and ct2[0] == 0 and ct2[1] == 2 for ct1, ct2 in l)
l

[((3, 9), (0, 2)), ((4, 8), (0, 2)), ((4, 10), (0, 2)), ((5, 5), (0, 2))]

# Case $d=4$ (Lemma 6.3)

Identify possible cases of an intermediate subfield for $d=4$ (Lemma 6.2).

In [25]:
for (d, g, g1) in [(4,3,9), (4,2,5)]:
    for (a,b,_1,_2) in candidates[d,g,g1]:
        for i in range(len(df)):
            if df.loc[i, "d"] == 2 and df.loc[i, "g"] == g and \
            df.loc[i, "g'"] == 2*g-1 and eval(df.loc[i, "Counts of C"]) == a:
                for j in range(len(df)):
                    if df.loc[j, "d"] == 2 and df.loc[j, "g"] == 2*g-1 and \
                    df.loc[j, "g'"] == g1 and \
                    df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                    eval(df.loc[j, "Counts of C'"]) == b:
                        print(d,g,g1,a,b)
                        assert g == 2


4 2 5 [2, 8, 8, 24, 52, 56, 100, 256, 476, 968, 2180, 4272, 8140] [0, 0, 12, 16, 60, 24, 84, 160, 444, 840, 2244, 4144, 8268]


Subcase $(g,g') = (3,9)$ and the quadratic resolvent is constant.

In [26]:
l = [((a[0], a[1], a[2], a[3]), (b[0], b[1], b[2], b[3])) for (a,b,c2,c4) in candidates[4,3,9] if c4%3 == 0]
assert all(ct1[1] == 9 and ct2[1] == 0 for ct1, ct2 in l)
assert len(l) == 1
l

[((5, 9, 8, 25), (0, 0, 0, 28))]

In [27]:
ct1, ct2 = l[0]
assert not splitting_sequences(ct1, ct2,4,alternating=True)

Subcase $(g,g') = (2,5)$ and the quadratic resolvent is constant.

In [28]:
l = [(a[0], b[0]) for (a,b,c2,c4) in candidates[4,2,5] if c4%3 == 0]
assert all(ct2 == 1 for ct1, ct2 in l)
l

[(5, 1)]

Subcase $(g,g') = (3,9)$ and the quadratic resolvent is purely geometric.

In [29]:
l = [((a[0], a[1]), (b[0], b[1])) for (a,b,_1,_2) in candidates[4,3,9]]
assert all(ct1[0] in [5,6] and ct2[0] == 0 and ct2[1] <= 2 for ct1, ct2 in l)
l

[((5, 9), (0, 0)),
 ((6, 8), (0, 0)),
 ((6, 8), (0, 2)),
 ((6, 8), (0, 0)),
 ((6, 8), (0, 2))]

Subcase $(g,g') = (2,5)$ and the quadratic resolvent is purely geometric.

In [30]:
l = [((a[0], a[1]), (b[0], b[1]), c2, c4) for (a,b,c2,c4) in candidates[4,2,5] if c2%2 == 0]
assert all(ct1[0] in [2,4] and c2%3 != 0 for ct1, ct2, c2, c4 in l)
l

[((2, 8), (0, 0), 4, 40),
 ((4, 4), (0, 4), 8, 16),
 ((4, 4), (1, 1), 8, 16),
 ((4, 8), (0, 8), 10, 40)]

# Case $d=5$ (Lemma 7.2)

Compile a list of all cases that need to be treated.

In [31]:
all_cases = [(a[:7], b[:7], c2, c4) for (a,b,c2,c4) in candidates[5,2,6]]
case_log = {(tuple(ct1), tuple(ct2), c2, c4): False for ct1, ct2, c2, c4 in all_cases}
all_cases

[([3, 5, 9, 33, 33, 65, 129], [0, 0, 0, 20, 15, 90, 105], 5, 25),
 ([3, 7, 9, 31, 33, 43, 129], [0, 0, 9, 8, 30, 33, 168], 6, 36),
 ([4, 8, 10, 24, 14, 56, 158], [0, 0, 15, 20, 20, 45, 175], 10, 40),
 ([4, 8, 10, 24, 14, 56, 158], [0, 6, 0, 18, 0, 60, 210], 10, 40),
 ([4, 10, 7, 18, 24, 55, 172], [0, 2, 15, 18, 40, 23, 140], 11, 55),
 ([5, 9, 5, 17, 25, 81, 145], [0, 6, 3, 34, 30, 63, 126], 15, 45),
 ([5, 9, 5, 17, 25, 81, 145], [0, 10, 0, 10, 25, 70, 140], 15, 45),
 ([6, 6, 9, 10, 36, 87, 90], [1, 3, 7, 27, 41, 69, 71], 19, 19)]

In [32]:
def certify(ct1, ct2, case_log, alternating=False):
    for long1, long2, c2, c4 in case_log:
        if long1[:len(ct1)] == tuple(ct1) and long2[:len(ct2)] == tuple(ct2):
            if alternating:
                assert c2%2 == 1
            case_log[long1, long2, c2, c4] = True

Assuming that $d=5$, convert a sequence of splittings to the traces of $B_1$ and $B_2$.

In [33]:
def splittings_to_B1_B2(ct1, ct2, s, return_both=False):
    d = {(5,): [(5,1), (5,) * 4],
         (3,1,1): [(3,3), (3,) * 6 + (1,) * 2],
         (2,2,1): [(2,) * 2 + (1,) * 2, (2,) * 10],
         (1,1,1,1,1): [(1,) * 6, (1,) * 20]}
    n = len(s)
    ct1p = place_count_from_point_count(ct1, n)
    ct2p = place_count_from_point_count(ct2, n)
    ct3p = [0 for _ in range(n)]
    ct4p = [0 for _ in range(n)]
    for i in range(n):
        for p in s[i]:
            pt = tuple(p)
            for j in d[pt][0]:
                if (i+1)*j <= n:
                    ct3p[(i+1)*j-1] += 1
            for j in d[pt][1]:
                if (i+1)*j <= n:
                    ct4p[(i+1)*j-1] += 1
    ct3 = point_count_from_place_count(ct3p, n)
    ct4 = point_count_from_place_count(ct4p, n)
    trA = [ct1[i] - ct2[i] for i in range(n)]
    trB1 = [ct1[i] - ct3[i] for i in range(n)]
    trB2 = [ct1[i] - ct4[i] - 2*trA[i] - trB1[i] for i in range(n)]
    return trB1, trB2

Subcase $(\#C(\mathbb{F}_2); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{32})) = (4; 0,6,0,18,0)$.

In [34]:
ct1, ct2 = [4,8,10,24,14],[0,6,0,18,0]
assert not splitting_sequences(ct1, ct2, 5)
certify(ct1, ct2, case_log)

Subcase $(\#C(\mathbb{F}_2), \#C(\mathbb{F}_4); \#C'(\mathbb{F}_2), \#C'(\mathbb{F}_4)) = (6,6;1,3)$.

In [35]:
ct1, ct2 = [6,6],[1,3]
assert not splitting_sequences(ct1, ct2,5,alternating=True)
certify(ct1, ct2, case_log, alternating=True)

Subcase $(\#C(\mathbb{F}_2), \dots, \#C(\mathbb{F}_8); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_8)) = (5,9,5;0,6,3)$.

In [36]:
ct1, ct2 = [5,9,5],[0,6,3]
assert not splitting_sequences(ct1, ct2, 5,alternating=True)
certify(ct1, ct2, case_log, alternating=True)

Subcase $(\#C(\mathbb{F}_2), \dots, \#C(\mathbb{F}_8); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_8)) = (4,10,7;0,2,15)$,

In [37]:
ct1, ct2 = [4,10,7],[0,2,15]
l = splitting_sequences(ct1,ct2,5,alternating=True)
assert all(s[2] == ([1,1,1,1,1],) for s in l)
certify(ct1, ct2, case_log, alternating=True)
l

[[([5], [5], [5], [5]), ([5], [5], [2, 2, 1]), ([1, 1, 1, 1, 1],)]]

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{32}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{32}))  = (3, 5, 9, 33, 33; 0, 0, 0, 20, 15)$.

In [38]:
ct1, ct2 = [3,5,9,33,33],[0,0,0,20,15]
l = splitting_sequences(ct1,ct2,5,alternating=True)
print(set(s[0] for s in l))
print(set(s[1] for s in l))
print(set(s[2] for s in l))
print(set(s[4] for s in l))

{([5], [5], [5])}
{([5],)}
{([5], [5])}
{([5], [5], [5], [5], [5], [5])}


In [39]:
tmp = [splittings_to_B1_B2(ct1, ct2, s) for s in l]
assert all(trB2[:3] == [-3, -5, -9] for trB1, trB2 in tmp)
tmp

[([0, 0, 0, -20, -15], [-3, -5, -9, -53, -48]),
 ([0, 0, 0, 4, -15], [-3, -5, -9, -13, -48]),
 ([0, 0, 0, -8, -15], [-3, -5, -9, 7, -48]),
 ([0, 0, 0, -20, -15], [-3, -5, -9, 27, -48])]

In [40]:
tmp2 = set(trace_from_weil_poly(u, 5)[4] for u in weil_polys_from_traces(P, 12, 2, [-3, -5, -9]))
assert not any(trB2[4] in tmp2 for trB1, trB2 in tmp)
tmp2

{-13, -8, -3, 2, 7, 12, 17, 22, 27, 32, 37, 47, 57}

In [41]:
certify(ct1, ct2, case_log, alternating=True)

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{8}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{8}))  = (4, 8, 10; 0, 0, 15)$,

In [42]:
ct1, ct2 = [4,8,10],[0,0,15]
set(s[0] for s in splitting_sequences(ct1,ct2,5))

{([5], [5], [5], [5])}

In [43]:
l = splitting_sequences(ct1,ct2,5,alternating=True)
l

[[([5], [5], [5], [5]), ([5], [5]), ([5], [1, 1, 1, 1, 1])]]

In [44]:
tmp = [splittings_to_B1_B2(ct1,ct2, s) for s in l]
tmp

[([0, 0, -15], [-4, -8, -25])]

In [45]:
assert all(not weil_polys_from_traces(P, 12, 2, trB2) for trB1, trB2 in tmp)

In [46]:
certify(ct1, ct2, case_log)

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{8}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{8}))  = (5, 9, 5; 0, 10, 0)$.

In [47]:
ct1, ct2 = [5,9,5],[0,10,0]
splitting_sequences(ct1,ct2,5)

[[([5], [5], [5], [5], [5]), ([5], [1, 1, 1, 1, 1]), ()],
 [([5], [5], [5], [5], [5]), ([3, 2], [1, 1, 1, 1, 1]), ()],
 [([5], [5], [5], [5], [5]), ([3, 1, 1], [2, 1, 1, 1]), ()]]

In [48]:
l = splitting_sequences(ct1,ct2,5,alternating=True)
l

[[([5], [5], [5], [5], [5]), ([5], [1, 1, 1, 1, 1]), ()]]

In [49]:
tmp = [splittings_to_B1_B2(ct1,ct2, s) for s in l]
tmp

[([0, -10, 0], [-5, -19, -5])]

In [50]:
assert all(not weil_polys_from_traces(P, 12, 2, trB2) for trB1, trB2 in tmp)

In [51]:
certify(ct1, ct2, case_log, alternating=True)

Subcase $(\#C(\mathbb{F}_2), \dots, \#C(\mathbb{F}_{128}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{128})) = (3,7,9,31,33,43,129;0,0,9,8,30,33,168)$.

In [52]:
ct1, ct2 = [3,7,9,31,33,43,129],[0,0,9,8,30,33,168]
l = splitting_sequences(ct1[:3],ct2[:3],5)
set([s[0] for s in l])

{([5], [5], [5])}

In [53]:
l = splitting_sequences(ct1,ct2,5,alternating=True)
set([s[1] for s in l]), set([s[2] for s in l]), set([s[3] for s in l]), set([s[4] for s in l]), set([s[5] for s in l])

({([5], [5])},
 {([3, 1, 1], [2, 2, 1])},
 {([5], [5], [5], [5], [2, 2, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [5], [3, 1, 1])},
 {([5], [5], [5], [2, 2, 1], [2, 2, 1], [2, 2, 1]),
  ([5], [5], [5], [5], [3, 1, 1], [2, 2, 1])},
 {([5], [5], [5], [2, 2, 1], [2, 2, 1]), ([5], [5], [5], [5], [3, 1, 1])})

In [54]:
tmp = [splittings_to_B1_B2(ct1, ct2, s) for s in l]
tmp2 = [(trB1, trB2) for (trB1, trB2) in tmp if weil_polys_from_traces(P, 12, 2, trB2)]
tmp3 = [trB1 for (trB1, trB2) in tmp2]
tmp4 = [trB2 for (trB1, trB2) in tmp2]
tmp3, tmp4

([[0, 0, 0, -8, -30, -24, 42],
  [0, 0, 0, -8, -30, -24, -42],
  [0, 0, 0, -8, -30, -24, -126]],
 [[-3, -7, 3, -7, -3, -19, 25],
  [-3, -7, 3, -7, -3, -19, 25],
  [-3, -7, 3, -7, -3, -19, 25]])

In [55]:
assert not any(weil_polys_from_traces(P, 10, 2, trB1) for trB1 in tmp3)

In [56]:
certify(ct1, ct2, case_log)

Check that we have handled all subcases.

In [57]:
assert all(case_log.values())

# Case $d=6$ (Lemma 8.3)

Identify possible cases of an intermediate subfield for $d=6$ (Lemma 8.2).

In [58]:
for (a,b,_1,_2) in candidates[6,2,7]:
    for i in range(len(df)):
        if df.loc[i, "d"] == 2 and df.loc[i, "g"] == 2 and \
        df.loc[i, "g'"] == 3 and eval(df.loc[i, "Counts of C"]) == a:
            for j in range(len(df)):
                if df.loc[j, "d"] == 3 and df.loc[j, "g"] == 3 and \
                df.loc[j, "g'"] == 7 and \
                df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                eval(df.loc[j, "Counts of C'"]) == b:
                    print(a,b)
                    assert (b[3] == 8)

[4, 8, 10, 24, 14, 56, 158, 256, 550, 968, 1918, 4272, 8246] [0, 0, 6, 8, 30, 24, 126, 192, 582, 840, 2046, 4400, 8502]


In [59]:
for (a,b,_1,_2) in candidates[6,2,7]:
    for i in range(len(df)):
        if df.loc[i, "d"] == 3 and df.loc[i, "g"] == 2 and \
        df.loc[i, "g'"] == 4 and eval(df.loc[i, "Counts of C"]) == a: 
            for j in range(len(df)):
                if df.loc[j, "d"] == 4 and df.loc[j, "g"] == 2 and \
                df.loc[j, "g'"] == 7 and \
                df.loc[i, "Counts of C'"] == df.loc[j, "Counts of C"] and \
                eval(df.loc[j, "Counts of C'"]) == b:
                    print(a,b)
                    assert False

Assuming $d=6$, convert a splitting sequence into the traces of the Prym for the sextic twin. 
If `S6 == False`, assume that $G \not\subseteq \mathrm{PGL}(2,5)$, so that the sextic twin becomes reducible.

In [60]:
def splittings_to_sextic_twin(ct1, s, S6=True):
    n = len(ct1)
    d = {(6,): [3,2,1],
         (3,2,1): [6],
         (4,1,1): [4,1,1],
         (2,2,2): [2,1,1,1,1],
         (2,1,1,1,1): [2,2,2],
         (5,1): [5,1],
         (4,2): [4,2],
         (3,3): [3,1,1,1],
         (3,1,1,1): [3,3],
         (2,2,1,1): [2,2,1,1],
         (1,1,1,1,1,1): [1,1,1,1,1,1]}
    s2 = [[d[tuple(p)] for p in s[i]] for i in range(len(s))]
    ct2p = place_count_from_splittings(s2)
    ct2 = point_count_from_place_count(ct2p, n)
    if S6:
        return [ct1[i] - ct2[i] for i in range(n)]
    return [2*ct1[i] - ct2[i] for i in range(n)]

Remove subcases based on the reduced resultant.

In [61]:
l = []
for (a,b,c2,c4) in candidates[6,2,7]:
    u1 = weil_poly_from_point_count(a, 2)
    u2 = weil_poly_from_point_count(b, 7)
    v1 = u1.trace_polynomial()[0]
    v2 = (u2//u1).trace_polynomial()[0]
    if modified_reduced_resultant(v1, v2)%3 != 0:
        l.append((a, b, c2, c4))
[(a[:6], b[:6], c2, c4) for (a,b,c2,c4) in l]

[([5, 5, 17, 9, 25, 65], [0, 0, 3, 12, 35, 87], 13, 13),
 ([5, 9, 5, 17, 25, 81], [0, 2, 9, 2, 30, 71], 15, 45),
 ([5, 9, 5, 17, 25, 81], [0, 4, 0, 12, 15, 106], 15, 45),
 ([6, 6, 9, 10, 36, 87], [0, 0, 15, 20, 30, 87], 19, 19)]

Identify remaining cases.

In [62]:
all_cases = [(a[:6], b[:6], c2, c4) for (a,b,c2,c4) in candidates[6,2,7] if (a,b,c2,c4) not in l]
case_log = {(tuple(ct1), tuple(ct2), c2, c4): False for ct1, ct2, c2, c4 in all_cases}
all_cases

[([4, 8, 10, 24, 14, 56], [0, 0, 6, 8, 30, 24], 10, 40),
 ([5, 5, 17, 9, 25, 65], [0, 0, 12, 4, 15, 90], 13, 13),
 ([5, 5, 17, 9, 25, 65], [0, 2, 3, 10, 30, 47], 13, 13),
 ([5, 7, 11, 15, 15, 91], [0, 2, 6, 10, 5, 116], 14, 28),
 ([5, 9, 5, 17, 25, 81], [1, 1, 1, 1, 41, 49], 15, 45),
 ([6, 6, 9, 10, 36, 87], [0, 2, 3, 14, 35, 131], 19, 19),
 ([6, 6, 9, 10, 36, 87], [0, 4, 3, 12, 30, 91], 19, 19)]

Subcases with the quadratic resolvent constant which require only consideration of splitting sequences.

In [63]:
ct1, ct2 = [5,5], [0, 2]
assert not splitting_sequences(ct1, ct2, 6, alternating=True)
certify(ct1, ct2, case_log)

In [64]:
ct1, ct2 = [5,9], [1, 1]
assert not splitting_sequences(ct1, ct2, 6, alternating=True)
certify(ct1, ct2, case_log)

In [65]:
ct1, ct2 = [6,6], [0,2]
assert not splitting_sequences(ct1, ct2, 6, alternating=True)
certify(ct1, ct2, case_log)

In [66]:
ct1, ct2 = [6,6], [0,4]
assert not splitting_sequences(ct1, ct2, 6, alternating=True)
certify(ct1, ct2, case_log)

In [67]:
ct1, ct2 = [4, 8,10,24,14,56], [0, 0, 6, 8, 30,24]
splitting_sequences(ct1[:3], ct2[:3], 6, alternating=True)

[[([6], [6], [6], [6]), ([4, 2], [4, 2]), ([6], [4, 1, 1])],
 [([6], [6], [6], [6]), ([4, 2], [4, 2]), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6]), ([4, 2], [4, 2]), ([3, 2, 1], [3, 2, 1])],
 [([6], [6], [6], [6]), ([4, 2], [3, 3]), ([6], [4, 1, 1])],
 [([6], [6], [6], [6]), ([4, 2], [3, 3]), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6]), ([4, 2], [3, 3]), ([3, 2, 1], [3, 2, 1])],
 [([6], [6], [6], [6]), ([3, 3], [3, 3]), ([6], [4, 1, 1])],
 [([6], [6], [6], [6]), ([3, 3], [3, 3]), ([4, 1, 1], [2, 2, 2])],
 [([6], [6], [6], [6]), ([3, 3], [3, 3]), ([3, 2, 1], [3, 2, 1])]]

In [68]:
assert not splitting_sequences(ct1, ct2, 6, alternating=True)
certify(ct1, ct2, case_log)

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{32}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{32})) = (5,7,11, 15,15; 0, 2, 6, 10,5)$ and quadratic resolvent constant.

In [69]:
ct1, ct2 = [5,7,11,15,15], [0,2,6,10,5]
l = splitting_sequences(ct1[:3], ct2[:3], 6, alternating=True)
set([s[0] for s in l])

{([6], [6], [6], [6], [6])}

In [70]:
l = splitting_sequences(ct1[:5], ct2[:5], 6, alternating=True)
l = [s for s in l if [4,1,1] not in s[2]]
print(set([s[1] for s in l]))
print(set([s[2] for s in l]))
print(set([s[3] for s in l]))
print(set([s[4] for s in l]))

{([5, 1],)}
{([3, 2, 1], [3, 2, 1])}
{([3, 3], [2, 2, 1, 1]), ([4, 2], [2, 2, 1, 1]), ([5, 1], [5, 1])}
{([3, 2, 1], [2, 2, 2]), ([6], [3, 2, 1])}


In [71]:
tmp = set([tuple(splittings_to_sextic_twin(ct1[:4], s)) for s in l])
tmp

{(0, -10, -9, -22), (0, -10, -9, -10)}

In [72]:
assert not any(weil_polys_from_traces(P, 10, 2, u) for u in tmp)

In [73]:
certify(ct1, ct2, case_log)

Subcase $(\#C(\mathbb{F}_2),\dots \#C(\mathbb{F}_{64}); \#C'(\mathbb{F}_2), \dots, \#C'(\mathbb{F}_{64})) = (5,5,17,9,25,65; 0,0,12,4,15,90)$ and quadratic resolvent constant.

In [74]:
ct1, ct2 = [5,5,17,9,25,65], [0,0,12,4,15,90]
l = splitting_sequences(ct1[:4], ct2[:4], 6, alternating=True)
set([s[0] for s in l])

{([6], [6], [6], [6], [6])}

In [75]:
l = splitting_sequences(ct1[:5], ct2[:5], 6, alternating=True)
l = [s for s in l if [2,1,1,1,1] not in s[2] and s[2].count([4,1,1]) <= 1]
print(set([s[1] for s in l]))
print(set([s[2] for s in l]))
print(set([s[3] for s in l]))
print(set([s[4] for s in l]))

{()}
{([6], [4, 1, 1], [3, 2, 1], [3, 2, 1]), ([4, 1, 1], [3, 2, 1], [3, 2, 1], [2, 2, 2]), ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1])}
{([5, 1],)}
{([6], [3, 2, 1], [3, 2, 1], [3, 2, 1]), ([6], [6], [4, 1, 1], [3, 2, 1]), ([6], [4, 1, 1], [3, 2, 1], [2, 2, 2]), ([3, 2, 1], [3, 2, 1], [3, 2, 1], [2, 2, 2]), ([4, 1, 1], [3, 2, 1], [2, 2, 2], [2, 2, 2])}


In [76]:
tmp = set(tuple(splittings_to_sextic_twin(ct1[:5], s)) for s in l)
tmp

{(0, -10, -21, -10, -30),
 (0, -10, -21, -10, -15),
 (0, -10, -21, -10, 0),
 (0, -10, -21, -10, 15),
 (0, -10, -12, -10, -30),
 (0, -10, -12, -10, -15),
 (0, -10, -12, -10, 0),
 (0, -10, -12, -10, 15),
 (0, -10, -3, -10, -30),
 (0, -10, -3, -10, -15),
 (0, -10, -3, -10, 0),
 (0, -10, -3, -10, 15)}

In [77]:
tmp2 = [trace_from_weil_poly(u, 6) for trB in tmp for u in weil_polys_from_traces(P, 10, 2, list(trB))]
assert len(tmp2) == 1
tmp2

[[0, -10, -3, -10, 15, 23]]

In [78]:
l = [s for s in l if splittings_to_sextic_twin(ct1[:5], s) == tmp2[0][:5]]
l

[[([6], [6], [6], [6], [6]),
  (),
  ([3, 2, 1], [3, 2, 1], [3, 2, 1], [3, 2, 1]),
  ([5, 1],),
  ([6], [3, 2, 1], [3, 2, 1], [3, 2, 1])]]

In [79]:
l2 = splitting_sequences(ct1, ct2, 6, alternating=True, seed=l)
assert all(splittings_to_sextic_twin(ct1,s) != tmp2[0] for s in l2)

In [80]:
certify(ct1, ct2, case_log)

Check that all cases with the quadratic resolvent constant have been settled.

In [81]:
assert all(case_log.values())

Subcases with the quadratic resolvent purely geometric.

In [82]:
counts = [(long1[:5], long2[:5]) for long1, long2, c2, c4 in case_log if c2%2 == 0]
counts

[((4, 8, 10, 24, 14), (0, 0, 6, 8, 30)),
 ((5, 7, 11, 15, 15), (0, 2, 6, 10, 5))]

In [83]:
PGL25_avoid = [[3,2,1],[4,2],[2,1,1,1,1],[3,1,1,1]]
for (ct1, ct2) in counts:
    print("Point counts: {}, {}".format(ct1, ct2))
    l = splitting_sequences(ct1[:3], ct2[:3], 6)
    l = [s for s in l if splittings_to_quadratic_resolvent(ct1[:3], s)[0] <= 2]
    print("Splittings in degree 1: {}".format(set(s[0] for s in l)))
    tmp = set(splittings_to_quadratic_resolvent(ct1[:3], s)[0] for s in l)
    assert len(tmp) == 1
    trB = list(tmp)[0]
    tmp = weil_polys_from_traces(P, 2, 2, [trB])
    assert len(tmp) == 1
    trB = trace_from_weil_poly(tmp[0], 4)
    print("Traces of B: {}".format(trB))
    l = [s for s in l if splittings_to_quadratic_resolvent(ct1[:3], s) == tuple(trB[:3])]
    print("Splittings in degree 2: {}".format(set(s[1] for s in l)))
    print("Splittings in degree 3: {}".format(set(s[2] for s in l)))
    trB1 = [splittings_to_sextic_twin(ct1[:3], s) for s in l]
    print("Traces of B1 (based on splitting): {}".format([set(t[i] for t in trB1) for i in range(3)]))
    lprime = [s for s in l if not any(i in s[2] for i in PGL25_avoid)]
    trB1prime = [splittings_to_sextic_twin(ct1[:3], s, S6=False) for s in lprime]
    print("Traces of B1' (based on splitting): {}".format([set(t[i] for t in trB1prime) for i in range(3)]))
    trB1 = [trace_from_weil_poly(t, 5) for t1 in trB1 for t in weil_polys_from_traces(P, 10, 2, t1)]
    print("Traces of B1 (by exhaustion): {}".format([set(t[i] for t in trB1) for i in range(4)]))
    trB1prime = [trace_from_weil_poly(t, 5) for t1 in trB1prime for t in weil_polys_from_traces(P, 8, 2, t1)]
    print("Traces of B1' (by exhaustion): {}".format([set(t[i] for t in trB1prime) for i in range(4)]))
    tmp = [s for s in splitting_sequences(ct1[:4], ct2[:4], 6, seed=l)]
    l1 = [s for s in tmp if splittings_to_sextic_twin(ct1[:4], s) in [t[:4] for t in trB1]]
    print("Splittings in degree 4 (S6): {}".format(set([s[3] for s in l1])))
    trB1 = [t1 for t1 in trB1 if any(splittings_to_sextic_twin(ct1[:4], s) == t1[:4] for s in l1)]
    print("Traces of B1 (based on splitting): {}".format([set(t[i] for t in trB1) for i in range(4)]))
    tmp = [s for s in splitting_sequences(ct1[:4], ct2[:4], 6, seed=lprime, avoid=PGL25_avoid)]
    l1prime = [s for s in tmp if splittings_to_sextic_twin(ct1[:4], s, S6=False) in [t[:4] for t in trB1prime]]
    if l1prime:
        print("Splittings in degree 4 (PGL25): {}".format(set([s[3] for s in l1prime])))
        trB1prime = [t1 for t1 in trB1prime if any(splittings_to_sextic_twin(ct1[:4], s, S6=False) == t1[:4] for s in l1prime)]
        print("Traces of B1' (based on splitting): {}".format([set(t[i] for t in trB1prime) for i in range(4)]))
    else: 
        print("Splittings in degree 4 (PGL25): none")
    tmp = [s for s in splitting_sequences(ct1[:5], ct2[:5], 6, seed=l1)]
    l1 = [s for s in tmp if splittings_to_sextic_twin(ct1[:5], s) in trB1]
    assert not l1
    print("Splittings in degree 5 (S6): none")
    if l1prime:
        tmp = [s for s in splitting_sequences(ct1[:5], ct2[:5], 6, seed=l1prime, avoid=PGL25_avoid)]
        l1prime = [s for s in tmp if splittings_to_sextic_twin(ct1[:5], s, S6=False) in trB1prime]
        assert not l1prime
        print("Splittings in degree 5 (PGL25): none")
    print("This case complete!")
    print("")

Point counts: (4, 8, 10, 24, 14), (0, 0, 6, 8, 30)
Splittings in degree 1: {([6], [6], [6], [3, 3])}
Traces of B: [2, 0, -4, -8]
Splittings in degree 2: {([6], [6]), ([6], [2, 2, 2]), ([2, 2, 2], [2, 2, 2])}
Splittings in degree 3: {([4, 2], [3, 3]), ([4, 2], [4, 2]), ([3, 3], [3, 3])}
Traces of B1 (based on splitting): [{-2}, {-8, -14, -20}, {-8, -26, -17}]
Traces of B1' (based on splitting): [{2}, {0, -6, -12}, {-16}]
Traces of B1 (by exhaustion): [{-2}, {-8}, {-8}, {0, 4, 8, 12, 16, -4}]
Traces of B1' (by exhaustion): [{2}, {0}, {-16}, {-8, -4}]
Splittings in degree 4 (S6): {([5, 1], [4, 2], [4, 2], [3, 2, 1]), ([4, 2], [4, 2], [3, 2, 1], [3, 2, 1]), ([6], [4, 2], [3, 2, 1], [3, 2, 1])}
Traces of B1 (based on splitting): [{-2}, {-8}, {-8}, {0, -4}]
Splittings in degree 4 (PGL25): {([6], [5, 1], [5, 1], [2, 2, 2]), ([6], [6], [2, 2, 2], [2, 2, 1, 1]), ([5, 1], [5, 1], [3, 3], [3, 3]), ([6], [6], [4, 1, 1], [2, 2, 2]), ([6], [6], [4, 1, 1], [3, 3]), ([6], [6], [3, 3], [2, 2, 1, 1])}
T

# Case $d=7$ (Lemma 9.1)

In [84]:
(d, g, g1) = (7, 2, 8)
candidates[d, g, g1]

[([5, 7, 11, 15, 15, 91, 131, 255, 479, 987, 2227, 3999, 8143],
  [0, 0, 0, 0, 0, 84, 133, 336, 378, 980, 2310, 4032, 8008],
  14,
  28)]

In [85]:
u = weil_poly_from_point_count(candidates[d, g, g1][0][0], g); u

T^4 + 2*T^3 + 3*T^2 + 4*T + 4

In [86]:
v = weil_poly_from_point_count(candidates[d, g, g1][0][1], g1); v

T^16 - 3*T^15 + 2*T^14 + 14*T^10 - 23*T^9 + 13*T^8 - 46*T^7 + 56*T^6 + 128*T^2 - 384*T + 256

In [87]:
K1 = magma.NumberField(u)
a  = K1.gen(1)
O1 = K1.MaximalOrder()
O1a = O1.sub([a, 2/a])
assert O1a == O1
assert O1.ClassNumber() == 1

In [88]:
K2 = magma.NumberField(v//u)
b = K2.gen(1)
O2 = K2.MaximalOrder()
O2a = O2.sub([b, 2/b])
assert O2a == O2
assert O2.ClassNumber() == 1

In [89]:
K2s = K2.sage()
l = K2s.primes_above(7)
assert(len(l) == 1)
p = l[0]
z = cyclotomic_polynomial(7).roots(K2s)[0][0]
assert K2s(z-1).valuation(p) == 1
assert K2s(7).valuation(p) == 6

## Lemma 9.3

In [90]:
u = weil_poly_from_point_count([0,0,0,20,15,90], 6); u

T^12 - 3*T^11 + 2*T^10 + 5*T^8 - 12*T^7 + 16*T^6 - 24*T^5 + 20*T^4 + 32*T^2 - 96*T + 64

In [91]:
v = u.factor()[-1][0]; v

T^8 - 3*T^7 + 2*T^6 + T^4 + 8*T^2 - 24*T + 16

In [92]:
K2 = magma.NumberField(v)
b = K2.gen(1)
O2 = K2.MaximalOrder()
O2a = O2.sub([b, 2/b])
assert O2a == O2
assert O2.ClassNumber() == 1

In [93]:
K2s = K2.sage()
l = K2s.primes_above(5)
assert K2s(5).valuation(l[0]) == 4
assert K2s(5).valuation(l[1]) == 4
z = cyclotomic_polynomial(5).roots(K2s)[0][0]
assert K2s(z-1).valuation(l[0]) == 1
assert K2s(z-1).valuation(l[1]) == 1